# pip installs

In [0]:
!pip install websocket_client

     |████████████████████████████████| 204kB 5.1MB/s 


In [0]:
!pip install schedule

In [0]:
!pip install apscheduler

     |████████████████████████████████| 61kB 3.9MB/s 


In [0]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 4.9MB/s 


# Feeding Websocket Data to OHLCV DataFrame -----> DataBase

In [0]:
import psycopg2 as ps

POSTGRES_ADDRESS = '' ## End Point
POSTGRES_PORT = '' ##Port #
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = '' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information 
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
                 username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, 
                 ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))

## Creating Tables

In [0]:
conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()
cur.execute('''
  CREATE TABLE BTC_USD_OHLCV
  (
  time INTEGER NOT NULL,
  open FLOAT NOT NULL,
  high FLOAT NOT NULL,
  low FLOAT NOT NULL,
  close FLOAT NOT NULL,
  volume FLOAT NOT NULL
  )

''')

conn.commit()
print("Table created Successfully!")
conn.close()

Table created Successfully!


In [0]:
conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()
cur.execute('''
  CREATE TABLE ETH_USD_OHLCV
  (
  time INTEGER NOT NULL,
  open FLOAT NOT NULL,
  high FLOAT NOT NULL,
  low FLOAT NOT NULL,
  close FLOAT NOT NULL,
  volume FLOAT NOT NULL
  )

''')

conn.commit()
print("Table created Successfully!")
conn.close()

Table created Successfully!


In [0]:
conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()
cur.execute('''
  CREATE TABLE LTC_USD_OHLCV
  (
  time INTEGER NOT NULL,
  open FLOAT NOT NULL,
  high FLOAT NOT NULL,
  low FLOAT NOT NULL,
  close FLOAT NOT NULL,
  volume FLOAT NOT NULL
  )

''')

conn.commit()
print("Table created Successfully!")
conn.close()

Table created Successfully!


## Run Webscoket and DataBase

- Run as long as you want to collect data. Run for 10 minutes to get good amount of data

In [0]:
import websocket
import json
from pprint import pprint
import pandas as pd
import numpy as np
from apscheduler.schedulers.blocking import BlockingScheduler
import psycopg2 as ps
from sqlalchemy import create_engine

URL = "wss://ws-feed.pro.coinbase.com"

POSTGRES_ADDRESS = '' ## End Point
POSTGRES_PORT = '' ##Port #
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = '' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information 
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
                 username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, 
                 ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))

try:
    conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

pairs = ['BTC-USD', 'LTC-USD', 'ETH-USD']
trades = {pair: pd.DataFrame() for pair in pairs}
ohlcvs = {pair: pd.DataFrame() for pair in pairs}

table_names = {'BTC-USD':'BTC_USD_OHLCV', 'ETH-USD':'ETH_USD_OHLCV', 'LTC-USD':'LTC_USD_OHLCV'}

scheduler = BlockingScheduler()

def job(pair):
    
    global ohlcvs
    global trades
    
    df = trades[pair]
    
    df = df[['time', 'price', 'size', 'trade_id']]
    new_index = pd.DatetimeIndex(df['time'])
    df['size'] = df['size'].apply(lambda x: float(x))
    df = df.set_index(new_index).drop(columns='time')
    vol = df['size'].resample('1Min').sum()
    df['price'] = df['price'].astype(float)
    ohlc1m = df['price'].resample('1Min').ohlc()
    ohlcvs[pair] = pd.concat([ohlc1m, vol], axis=1)
    ohlcvs[pair] = ohlcvs[pair].reset_index()
    ohlcvs[pair]['time'] = pd.to_datetime(ohlcvs[pair]['time']).values.astype(np.int64) // 10**6 
    ohlcvs[pair]['time'] = ohlcvs[pair]['time'].apply(lambda x: int(x/1000))
    print('OHLCV Working', ohlcvs[pair] )
    
    
    conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)
    cur = conn.cursor()
    table_name = table_names[pair]
    cur.execute("SELECT time FROM "+ table_name +" order by time desc limit 1")
    
    cur1 = conn.cursor()
    table_name = table_names[pair]
    cur1.execute("SELECT time FROM "+ table_name)  
        
    try:  
      if (ohlcvs[pair]['time'].tolist()[-1]) != (cur.fetchall()[0][0]):
        print('---------ENTER ELIF------------')
  
        table_name = table_names[pair]
        test_data = ohlcvs[pair].iloc[-1:,:].values.tolist()[0]
        insert_query = "INSERT INTO {} (time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)".format(table_name)  
  
        cur.execute(insert_query, test_data)
        print('Inserted ELIF ROW!')
        conn.commit()
    except:
      pass
    
    if len(cur1.fetchall()) == 0:
      print("Started INSERT ROW")
      table_name = table_names[pair]
      test_data = ohlcvs[pair].iloc[-1:,:].values.tolist()[0]
      insert_query = "INSERT INTO {} (time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)".format(table_name)  
  
      cur1.execute(insert_query, test_data)
      print('Inserted ROW!')
      conn.commit()
      
    
def on_message(ws, message):
    """Callback executed when a message comes.
    Positional argument:
    message -- The message itself (string)
    """
    msg = json.loads(message)
    #pprint(msg)
    
    global trades
    
    for pair in trades:
        if msg['product_id'] == pair:
            trades[pair] = trades[pair].append(msg, ignore_index=True)
            
        try:
            scheduler.add_job(job(pair), 'interval', minutes=1)
            scheduler.start()
        except:
            pass
          
def on_open(socket):
    """Callback executed at socket opening.
    Keyword argument:
    socket -- The websocket itself
    """
    params = {
        "type": "subscribe",
        "channels": [{"name": "matches", "product_ids": ["BTC-USD", "ETH-USD", "LTC-USD"]}]
    }
    socket.send(json.dumps(params))
    
def main():
    """Main function."""
    ws = websocket.WebSocketApp(URL, on_open=on_open, on_message=on_message)
    ws.run_forever()
    
      
if __name__ == '__main__':
    main()

Connected!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


OHLCV Working          time     open     high      low    close      size
0  1571193420  8162.99  8162.99  8162.99  8162.99  0.012066
Started INSERT ROW
Inserted ROW!
OHLCV Working          time     open     high      low    close      size
0  1571193420  8162.99  8162.99  8162.99  8162.99  0.012066
OHLCV Working          time    open    high     low   close      size
0  1571193420  179.61  179.61  179.61  179.61  0.553992
Started INSERT ROW
Inserted ROW!
OHLCV Working          time     open     high      low    close      size
0  1571193420  8162.99  8162.99  8162.99  8162.99  0.012066
OHLCV Working          time   open   high    low  close      size
0  1571193420  54.91  54.91  54.91  54.91  1.742517
Started INSERT ROW
Inserted ROW!
OHLCV Working          time    open    high     low   close      size
0  1571193420  179.61  179.61  179.61  179.61  0.553992
OHLCV Working          time     open    high      low   close      size
0  1571193420  8162.99  8163.0  8162.99  8163.0  0.023891

In [0]:
import psycopg2 as ps
conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)

cur = conn.cursor()
cur.execute("SELECT time FROM BTC_USD_OHLCV")

results = cur.fetchall()
#print(len(results))
#results = (results.astype(int))
len(results)
conn.close()

In [0]:
import pandas as pd 

pairs = ['BTC-USD', 'LTC-USD', 'ETH-USD']
trades = {pair: pd.DataFrame() for pair in pairs}
ohlcvs = {pair: pd.DataFrame() for pair in pairs}

table_names = {'BTC-USD':'BTC_USD_OHLCV', 'ETH-USD':'ETH_USD_OHLCV', 'LTC-USD':'LTC_USD_OHLCV'}

conn = ps.connect(host=POSTGRES_ADDRESS,database=POSTGRES_DBNAME,user=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,port=POSTGRES_PORT)
cur = conn.cursor()
table_name = table_names['BTC-USD']
cur.execute("SELECT time FROM "+ table_name)
print(len(cur.fetchall()))

0
